In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sqlite3
from pprint import pprint as pp
import json
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook
from bokeh.models import ColumnDataSource, Line
import pandas_bokeh
import os

In [2]:
output_notebook()

Loading BokehJS ...

In [3]:
conn = sqlite3.connect('../db/2_Jun.db')

In [47]:
ndvi_df = pd.read_sql("select * from ndvi", conn, parse_dates=['date_time'])
ndvi_sorted = ndvi_df.sort_values('date_time')
evi_df = pd.read_sql("select * from evi", conn, parse_dates=['date_time'])
evi_sorted = evi_df.sort_values('date_time')
lai_df = pd.read_sql("select * from lai", conn, parse_dates=['date_time'])
lai_sorted = lai_df.sort_values('date_time')

In [48]:
ndvi_df.head(10)

,index,georegion,max,min,mean,median,std,var,pixels,25_percent,50_percent,75_percent,date_time,farm_visible,vis_moisture,e18,hay_field,cedar_hill
0,0,Farm boundary,0.568599,0.247206,0.390880,0.379591,0.055476,0.003078,2284,0.354278,0.379591,0.421109,2019-09-18 16:40:00,0,1,0,0,0
1,1,Paddock 1,0.523407,0.442752,0.479321,0.477222,0.019741,0.000390,45,0.463514,0.477222,0.493716,2019-09-18 16:40:00,0,1,0,0,0
2,2,Paddock 2,0.546131,0.465743,0.508847,0.509678,0.017560,0.000308,78,0.498761,0.509678,0.516892,2019-09-18 16:40:00,0,1,0,0,0
3,3,Paddock 3,0.568599,0.435613,0.494438,0.489674,0.028528,0.000814,43,0.477033,0.489674,0.511853,2019-09-18 16:40:00,0,1,0,0,0
4,4,Paddock 4,0.529527,0.397916,0.466227,0.468793,0.036696,0.001347,69,0.437097,0.468793,0.496718,2019-09-18 16:40:00,0,1,0,0,0
5,5,Paddock 5,0.492976,0.380699,0.446648,0.445614,0.025874,0.000669,63,0.431553,0.445614,0.469613,2019-09-18 16:40:00,0,1,0,0,0
6,6,Paddock 6,0.408355,0.327199,0.368069,0.369075,0.019381,0.000376,68,0.353622,0.369075,0.383773,2019-09-18 16:40:00,0,1,0,0,0
7,7,Paddock 7,0.421742,0.341956,0.383064,0.380444,0.022782,0.000519,63,0.361410,0.380444,0.401476,2019-09-18 16:40:00,0,1,0,0,0
8,8,Paddock 8,0.365578,0.311887,0.336527,0.336959,0.012258,0.000150,61,0.326374,0.336959,0.346756,2019-09-18 16:40:00,0,1,0,0,0
9,9,Paddock 9,0.381344,0.328929,0.348823,0.346540,0.013008,0.000169,54,0.338967,0.346540,0.357420,2019-09-18 16:40:00,0,1,0,0,0


In [62]:
graze_df = ndvi_sorted[(ndvi_sorted['georegion'] == 'Paddock 9') & (ndvi_sorted['vis_moisture'] == 0)]
graze_df.head(10)

,index,georegion,max,min,mean,median,std,var,pixels,25_percent,50_percent,75_percent,date_time,farm_visible,vis_moisture,e18,hay_field,cedar_hill
961,9,Paddock 9,0.603371,0.400731,0.471639,0.475533,0.044409,0.001972,54,0.431112,0.475533,0.502782,2019-04-09 16:48:00,1,0,1,1,1
793,9,Paddock 9,0.628889,0.451306,0.518419,0.520163,0.036397,0.001325,54,0.485460,0.520163,0.538604,2019-04-16 16:38:00,1,0,1,1,1
1409,9,Paddock 9,0.748023,0.544602,0.635439,0.636270,0.039713,0.001577,54,0.600956,0.636270,0.656435,2019-04-19 16:48:00,1,0,1,1,1
1185,9,Paddock 9,0.867971,0.685714,0.778348,0.775952,0.036815,0.001355,54,0.757162,0.775952,0.802301,2019-04-26 16:38:00,1,0,1,1,1
261,9,Paddock 9,0.888018,0.843478,0.871531,0.871429,0.009176,0.000084,54,0.866303,0.871429,0.879126,2019-05-31 16:39:00,1,0,1,1,1
1353,9,Paddock 9,0.893501,0.845411,0.880646,0.882396,0.009965,0.000099,54,0.878648,0.882396,0.888360,2019-06-08 16:48:00,1,0,1,1,1
569,9,Paddock 9,0.894677,0.849098,0.880347,0.882870,0.010644,0.000113,54,0.878584,0.882870,0.887681,2019-06-10 16:39:00,1,0,1,1,1
821,9,Paddock 9,0.789931,0.555831,0.720193,0.726360,0.044944,0.002020,54,0.700260,0.726360,0.751615,2019-07-08 16:48:00,1,0,1,1,1
205,9,Paddock 9,0.814382,0.613180,0.748632,0.753156,0.037462,0.001403,54,0.732864,0.753156,0.770295,2019-07-15 16:38:00,1,0,1,1,1
457,9,Paddock 9,0.829575,0.671907,0.768836,0.769329,0.026220,0.000687,54,0.755934,0.769329,0.784895,2019-07-20 16:39:00,1,0,1,1,1


In [84]:
output_file('Index-rain.html')

source = ColumnDataSource(graze_df)
precip_source = ColumnDataSource(valid_precip)

p = figure(x_axis_type="datetime", y_range=(0,1),
          title='Index data',
          x_axis_label='Date',
          y_axis_label='NDVI')
p.line('date_time', 'mean', source=source, legend_label='NDVI', color='green', line_width=2)
p.line('DATE', 'SMA', source=precip_source, legend_label='Rain')

show(p)

In [68]:
precip_df = pd.read_csv('../../data/Jan 19 - May 20 precip.csv', parse_dates=['DATE'])
precip_df

,STATION,NAME,DATE,PRCP,PRCP_ATTRIBUTES,SNOW,SNOW_ATTRIBUTES
0,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-01,0.00,"T,,W,2400",0.0,"T,,W"
1,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-02,0.00,"T,,W,2400",0.0,"T,,W"
2,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-03,0.00,",,W,2400",0.0,",,W"
3,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-04,0.00,",,W,2400",0.0,",,W"
4,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2019-01-05,0.00,",,W,2400",0.0,",,W"
...,...,...,...,...,...,...,...
494,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-09,0.00,"T,,D,2400",0.0,",,D"
495,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-10,0.25,",,D,2400",0.0,",,D"
496,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-11,0.00,"T,,D,2400",0.0,",,D"
497,USW00094822,"ROCKFORD GREATER ROCKFORD AIRPORT, IL US",2020-05-12,0.00,",,D,2400",0.0,",,D"


In [82]:
precip_df['SMA'] = precip_df.loc[:,['PRCP']].rolling(window=7).mean()
valid_precip = precip_df[precip_df['SMA'] >= 0]